<!-- ## Rag -->

In [2]:
from crewai import LLM
from dotenv import load_dotenv
import os
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

llm = LLM(
    model="llama-3.1-8b-instant",
    api_key= GROQ_API_KEY,temperature=0.7
)

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = LLM(
    model="huggingface/meta-llama/Meta-Llama-3.1-8B-Instruct",
	api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

## Chroma check

In [17]:
from crewai.utilities.paths import db_storage_path
import os

storage_path = db_storage_path()
print(f"CrewAI storage location: {storage_path}")

# সব ফোল্ডার দেখুন
if os.path.exists(storage_path):
    for item in os.listdir(storage_path):
        print(f"📁 {item}")

CrewAI storage location: /home/ahmed/snap/code/219/.local/share/Agent
📁 chroma.sqlite3
📁 chromadb-b6e1f73bd8c9480e42e90856fc17defb.lock
📁 .crewai_user.json
📁 ee9e4d22-5caf-4327-a98e-5574ad9bf99c
📁 latest_kickoff_task_outputs.db


In [18]:
import chromadb
from crewai.utilities.paths import db_storage_path
import os
import shutil

# CrewAI storage path খুঁজুন
storage_path = db_storage_path()
print(f"Storage path: {storage_path}")

# # সম্পূর্ণ storage ডিলিট করুন
# if os.path.exists(storage_path):
#     shutil.rmtree(storage_path)
#     print("✅ Storage deleted!")

Storage path: /home/ahmed/snap/code/219/.local/share/Agent


In [19]:
from crewai.utilities.paths import db_storage_path
import os

storage_path = db_storage_path()
print(f"CrewAI storage location: {storage_path}")

# সব ফোল্ডার দেখুন
if os.path.exists(storage_path):
    for item in os.listdir(storage_path):
        print(f"📁 {item}")

CrewAI storage location: /home/ahmed/snap/code/219/.local/share/Agent
📁 chroma.sqlite3
📁 chromadb-b6e1f73bd8c9480e42e90856fc17defb.lock
📁 .crewai_user.json
📁 ee9e4d22-5caf-4327-a98e-5574ad9bf99c
📁 latest_kickoff_task_outputs.db


## Slow query

In [ ]:
import os
from crewai import Agent, Crew, Task, LLM
from crewai.knowledge.source.pdf_knowledge_source import PDFKnowledgeSource


# ৩. PDF Knowledge Source তৈরি
pdf_source = PDFKnowledgeSource(
    file_paths=["MachineLearning.pdf"],
    chunk_size=500,
    chunk_overlap=200
)

# ৪. Agent তৈরি
agent = Agent(
    role="PDF Knowledge Expert",
    goal="Answer questions based on the provided PDF documents.",
    backstory="An AI assistant that extracts and provides information from PDFs.",
    llm=llm,
    knowledge_sources=[pdf_source],
    verbose=True
)

# ৫. Task তৈরি
task = Task(
    description="Answer this question: {input}",
    expected_output="A clear and concise answer from the PDF.",
    agent=agent
)

# ৬. Crew তৈরি
crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    embedder={
        "provider": "sentence-transformer",
        "config": {"model_name": "all-MiniLM-L6-v2"}
    }
)

# # ৭. রান করুন
# if __name__ == "__main__":
#     result = crew.kickoff(inputs={"input": "Should You Buy This Book?"})
#     print("\n=== Result ===")
#     print(result)
    
#     # স্টোরেজ পাথ চেক করুন
#     from crewai.utilities.paths import db_storage_path
#     print(f"\nStorage path: {db_storage_path()}")

## 💡 সংক্ষেপে:

| বিষয়     | PDFSearchTool                            | PDFKnowledgeSource                           |
| --------- | ---------------------------------------- | -------------------------------------------- |
| ধরন       | Tool (টুল)                               | Knowledge Source (জ্ঞান উৎস)                 |
| কাজের ধরন | Agent নিজে সিদ্ধান্ত নেয় কখন সার্চ করবে | স্বয়ংক্রিয়ভাবে প্রতিটি task এ context দেয় |
| গতি       | দ্রুত (শুধু query অনুযায়ী সার্চ)        | ধীর (সম্পূর্ণ PDF প্রসেস করে)                |
| ব্যবহার   | `tools=[pdf_tool]`                       | `knowledge_sources=[pdf_source]`             |


- PDFSearchTool: দ্রুত, question-specific, agent নিজে decide করে কখন সার্চ।

- PDFKnowledgeSource: ধীর, পুরো PDF process করে, প্রতিটি task automatically context পায়।


## Faster query

In [3]:
from crewai import Agent, Crew, Task, LLM
from crewai_tools import PDFSearchTool


# PDFSearchTool ব্যবহার করুন - এটি দ্রুত
pdf_tool = PDFSearchTool(
    pdf='knowledge/MachineLearning.pdf',
    config={
        "embedding_model": {
            "provider": "sentence-transformer",
            "config": {"model": "all-MiniLM-L6-v2"}
        },
        "vectordb": {
            "provider": "chromadb",
            "config": {}
        }
    }
)

/home/ahmed/miniconda3/envs/aiapp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# llm = LLM(
#     model="llama-3.1-8b-instant", # The provider prefix is 'groq/'
#     # Replace with your actual key
#     api_key= GROQ_API_KEY,temperature=0.7
# )

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = LLM(
    model="huggingface/meta-llama/Llama-3.1-8B-Instruct",
	api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

# from crewai import LLM

# llm = LLM(
#     model="gemini/gemini-2.0-flash",
#     # api_key=os.getenv("GEMINI_API_KEY")
#     api_key=""
# )

agent = Agent(
    role="PDF Knowledge Expert",
    goal="Answer questions based on PDF documents.",
    backstory="An AI assistant that extracts information from PDFs.",
    llm=llm,
    tools=[pdf_tool],  # tools হিসেবে দিন, knowledge_sources নয়
    # verbose=True
)

task = Task(
    description="Answer: {input}",
    expected_output="Short and clear answer from the PDF.",
    agent=agent
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    # verbose=True
)

result = crew.kickoff(inputs={"input": "Decision Tree Learning?"})

In [23]:
result.raw

'3.3\n\nDecision Tree Learning\n\nA decision tree is an acyclic graph that can be used to make decisions. In each branching\n\nnode of the graph, a speciﬁc feature j of the feature vector is examined. If the value of the\n\nfeature is below a speciﬁc threshold, then the left branch is followed; otherwise, the right\n\nbranch is followed. As the leaf node is reached, the decision is made about the class to which\n\nthe example belongs.\n\nAs the title of the section suggests, a decision tree can be learned from data.\n\n3.3.1\n\nProblem Statement\n\nLike previously, we have a collection of labeled examples; labels belong to the set {0, 1}. We\n\nwant to build a decision tree that would allow us to predict the class of an example given a\n\nfeature vector.\n\n3.3.2\n\nSolution\n\nThere are various formulations of the decision tree learning algorithm. In this book, we\n\nconsider just one, called ID3.\n\nThe optimization criterion, in this case, is the average log-likelihood:\n\n1\n\nN\n\

In [25]:
result = crew.kickoff(inputs={"input": "Semi-Supervised Learning?"})
result

CrewOutput(raw='1.2.3\n\nSemi-Supervised Learning\n\nIn semi-supervised learning, the dataset contains both labeled and unlabeled examples.\n\nUsually, the quantity of unlabeled examples is much higher than the number of labeled\n\nexamples. The goal of a semi-supervised learning algorithm is the same as the goal of\n\nthe supervised learning algorithm. The hope here is that using many unlabeled examples can\n\nhelp the learning algorithm to ﬁnd (we might say “produce” or “compute”) a better model2.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer: Semi-Supervised Learning?', name='Answer: Semi-Supervised Learning?', expected_output='Short and clear answer from the PDF.', summary='Answer: Semi-Supervised Learning?...', raw='1.2.3\n\nSemi-Supervised Learning\n\nIn semi-supervised learning, the dataset contains both labeled and unlabeled examples.\n\nUsually, the quantity of unlabeled examples is much higher than the number of labeled\n\nexamples. The goal of

### agent2 wikepedia

In [26]:
from crewai.tools import tool
from crewai_tools import SerperDevTool
import wikipedia

serper_search = SerperDevTool()

@tool("Wikipedia Search")
def wikipedia_search(query: str) -> str:
    """Search Wikipedia for information."""
    try:
        return wikipedia.summary(query, sentences=5)
    except:
        return "No results found."


from crewai import LLM

# llm = LLM(
#     model="gemini/gemini-2.5-flash-lite", # gemma-3-12b
#     api_key=""
# )

# from crewai import LLM

# llm = LLM(
#     provider="groq",
#     model="llama-3.1-8b-instant",
#     api_key=os.getenv("GROQ_API_KEY")
# )


In [ ]:
result = crew.kickoff(inputs={"input": "whats is agentic ai?"})


In [ ]:
result = crew.kickoff(inputs={"input": "whats is Linear Regression?"})


## single agent

In [3]:
llm = LLM(
    model = "huggingface/meta-llama/Llama-3.3-70B-Instruct",
    # model="huggingface/meta-llama/Llama-3.1-8B-Instruct",
    api_key= os.getenv("HUGGINGFACEHUB_API_TOKEN")
) 


In [4]:
from crewai import Agent, Crew, Task, LLM
from crewai_tools import PDFSearchTool


# PDFSearchTool ব্যবহার করুন - এটি দ্রুত
pdf_tool = PDFSearchTool(
    pdf='knowledge/MachineLearning.pdf',
    config={
        "embedding_model": {
            "provider": "sentence-transformer",
            "config": {"model": "all-MiniLM-L6-v2"}
        },
        "vectordb": {
            "provider": "chromadb",
            "config": {}
        }
    }
)

/home/ahmed/miniconda3/envs/aiapp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
custom_system_template = """You are {role}. {backstory}

Your goal is: {goal}

Rules:
- Always search and use the PDF content first.
- If the PDF does not contain relevant information,search Wikipedia.
- Do not add external knowledge beyond PDF or Wikipedia.
- Keep answers short, clear, and factual.
"""

custom_prompt_template = """Question: {input}

Instructions:
- First, try to answer using the PDF content.
- If the PDF does not contain relevant context,then go wikipedia_search tool.
- Do not include assumptions or extra explanations.
"""

agent = Agent(
    role="Research Assistant",
    goal="Answer user questions using PDF first, then Wikipedia as fallback",
    backstory="Prioritize PDF content, use Wikipedia strictly as fallback.",
    system_template=custom_system_template,
    prompt_template=custom_prompt_template,
    use_system_prompt=True,
    llm=llm,
    tools=[pdf_tool, wikipedia_search],
    verbose=True
)


task = Task(
    description="Answer this user input: {input} using PDF first, if not found use Wikipedia",
    expected_output="Short answer from PDF or Wikipedia",
    agent=agent
)

In [43]:
# =========================
    # Create Agent with Better Instructions
    # =========================
custom_system_template = """You are {role}. {backstory}

Your goal is: {goal}

STRICT RULES:
1. FIRST search the PDF for relevant content using the PDF tool
2. ONLY if PDF search returns NO results, then use Wikipedia
3. NEVER invent or add external knowledge
4. Always cite your source: (PDF) or (Wikipedia)
5. If no information found in either source, say: "I couldn't find this information in the PDF or Wikipedia."
"""

custom_prompt_template = """QUESTION: {input}

SEARCH INSTRUCTIONS:
1. Search PDF for relevant content
2. If PDF has content, use it
3. If PDF has NO content, search Wikipedia
4. If neither has content, state clearly

ANSWER FORMAT:
- Start with direct answer
- Include key points
- End with source: (PDF) or (Wikipedia)
"""

agent = Agent(
    role="PDF Knowledge Expert",
    goal="Answer questions accurately using PDF content first, Wikipedia only as fallback",
    backstory="Prioritize PDF content, use Wikipedia strictly as fallback.",
    system_template=custom_system_template,
    prompt_template=custom_prompt_template,
    use_system_prompt=True,
    llm=llm,
    tools=[pdf_tool,wikipedia_search],
    # verbose=True,
    max_iter=3,
    allow_delegation=False
)


# agent = Agent(
#     role="PDF Knowledge Expert",
#     goal="Answer questions based on PDF documents.",
#     backstory="An AI assistant that extracts information from PDFs.",
#     llm=llm,
#     tools=[pdf_tool],  # tools হিসেবে দিন, knowledge_sources নয়
#     # verbose=True
# )

task = Task(
    description="Answer: {input}",
    expected_output="Short and clear answer from the PDF.",
    agent=agent
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    # verbose=True
)

In [44]:
crew = Crew(
    agents=[agent],
    tasks=[task],
    # verbose=True,
    # stream=True
)

result = crew.kickoff(inputs={"input": "Semi-Supervised Learning?"})
result

CrewOutput(raw='To answer the question: "Semi-Supervised Learning?"\n\n**Direct Answer:**\nSemi-supervised learning is the field of study that focuses on the development of algorithms that learn from both labeled and unlabeled data.\n\n**Key Points:**\n\n- Semi-supervised learning combines supervised and unsupervised learning techniques.\n- It is used in scenarios where large amounts of labeled data are not available but unlabeled data is abundant.\n- The goal of semi-supervised learning is to reduce the need for labeled data and improve the overall performance of machine learning models.\n- Semi-supervised learning algorithms can be divided into two main categories: graph-based and manifold-based methods.\n\n**Source:** (Wikipedia)', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer: Semi-Supervised Learning?', name='Answer: Semi-Supervised Learning?', expected_output='Short and clear answer from the PDF.', summary='Answer: Semi-Supervised Learning?...', raw=

In [32]:
result

CrewOutput(raw='Page 8:\n\n1.2.3\n\nSemi-Supervised Learning\n\nIn semi-supervised learning, the dataset contains both labeled and unlabeled examples.\n\nUsually, the quantity of unlabeled examples is much higher than the number of labeled\n\nexamples. The goal of a semi-supervised learning algorithm is the same as the goal of\n\nthe supervised learning algorithm. The hope here is that using many unlabeled examples can\n\nhelp the learning algorithm to ﬁnd (we might say “produce” or “compute”) a better model2.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer: Semi-Supervised Learning?', name='Answer: Semi-Supervised Learning?', expected_output='Short and clear answer from the PDF.', summary='Answer: Semi-Supervised Learning?...', raw='Page 8:\n\n1.2.3\n\nSemi-Supervised Learning\n\nIn semi-supervised learning, the dataset contains both labeled and unlabeled examples.\n\nUsually, the quantity of unlabeled examples is much higher than the number of labeled\n\

In [45]:
result = crew.kickoff(inputs={"input": "Whats AI Agent?"})
result

CrewOutput(raw='To answer the question: "What\'s an AI Agent?"\n\n**Direct Answer:** \nAn intelligent agent is a program that can perform tasks that would typically require human intelligence.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer: Whats AI Agent?', name='Answer: Whats AI Agent?', expected_output='Short and clear answer from the PDF.', summary='Answer: Whats AI Agent?...', raw='To answer the question: "What\'s an AI Agent?"\n\n**Direct Answer:** \nAn intelligent agent is a program that can perform tasks that would typically require human intelligence.', pydantic=None, json_dict=None, agent='PDF Knowledge Expert', output_format=<OutputFormat.RAW: 'raw'>, messages=[{'role': 'user', 'content': 'You are PDF Knowledge Expert. Prioritize PDF content, use Wikipedia strictly as fallback.\n\nYour goal is: Answer questions accurately using PDF content first, Wikipedia only as fallback\n\nSTRICT RULES:\n1. FIRST search the PDF for relevant content using the